In [11]:

# coding: utf-8

# In[ ]:

import falcon
import sys, os

from fft import fft, ifft
from ip import ipFFT, inBufferFFT, ipiFFT, inBufferiFFT
from get_vector import get_fft, get_ifft
from time import time

sys.path.append('/home/xilinx')
os.environ['XILINX_XRT'] = '/usr'

if __name__ == "__main__":
    print("Entry:", sys.argv[0])
    print("System argument(s):", len(sys.argv))
    print("Start of \"" + sys.argv[0] + "\"")
    print("=================================================")
    print("Start comparing the [FFT] IP and function...")

    f_in = [0 for i in range(1024)]
    # Read the Input
    with open("FFT_in.txt", "r") as file:
        for i in range(1024):
            f_in[i] = float(file.readline())
    # Start the Python Function
    start_function = time()
    ans = fft(f_in)
    end_function = time()

    for i in range(1024):
        inBufferFFT[i] = f_in[i]
    
    # Start the FFT IP
    start_ip = time()
    ipFFT.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
    while (ipFFT.read(0x00) & 0x4) == 0x0:
        continue
    end_ip = time()

    ip_out = get_fft(inBufferFFT)
    
    # Compare
    error_cnt = 0
    for i in range(len(ans)):
        r = 7
        # Round to the 7th digit after the decimal point
        ip_real = round(ip_out[i].real, r)
        ans_real = round(ans[i].real, r)
        ip_imag = round(ip_out[i].imag, r)
        ans_imag = round(ans[i].imag, r)
        f = complex(ip_real, ip_imag)
        a = complex(ans_real, ans_imag)
        if (f != a):
            print(f"Expect: {ans[i]}, actual: {ip_out[i]}")
            error_cnt += 1
    if (error_cnt != 0):
        print(f"Failed!!, there are {error_cnt+1} errors")
    else:
        print("Passed!!")

    print(f"{'FFT python execution time:':20} {(end_function - start_function):>.10f} sec")
    print(f"{'FFT IP execution time:':20}     {(end_ip - start_ip):>.10f} sec")
    print(f"Acceleration rate: {round((end_function - start_function) / (end_ip - start_ip), 2)}")
    print("-------------------------------------------------")


    print("Start comparing the [iFFT] IP and function...")

    f_in = get_ifft(ans) # Re-construct the order of the FFT output

    # Start the Python Function
    start_function = time()
    ans = ifft(ans) # upload the golden to the output of iFFT function
    end_function = time()

    for i in range(1024):
        inBufferiFFT[i] = f_in[i]

    # Start the iFFT IP
    start_ip = time()
    ipiFFT.write(0x00, 0x01) # 0x00: bit 0: 0->1: start
    while (ipiFFT.read(0x00) & 0x4) == 0x0:
        continue
    end_ip = time()

    # Compare
    error_cnt = 0
    for i in range(len(ans)):
        r = 10
        if (round(inBufferiFFT[i], r) != round(ans[i], r)):
            print(f"Expect: {ans[i]}, actual: {inBufferiFFT[i]}")
            error_cnt += 1
    if (error_cnt != 0):
        print(f"Failed!!, there are {error_cnt+1} errors")
    else:
        print("Passed!!")

    print(f"{'iFFT python execution time:':20} {(end_function - start_function):>.10f} sec")
    print(f"{'iFFT IP execution time:':20}     {(end_ip - start_ip):>.10f} sec")
    print(f"Acceleration rate: {round((end_function - start_function) / (end_ip - start_ip), 2)}")
    print("=================================================")
    print("Exit process")
    


Entry: /usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py
System argument(s): 3
Start of "/usr/local/share/pynq-venv/lib/python3.8/site-packages/ipykernel_launcher.py"
Start comparing the [FFT] IP and function...
Passed!!
FFT python execution time: 0.0097510815 sec
FFT IP execution time:     0.0020282269 sec
Acceleration rate: 4.81
-------------------------------------------------
Start comparing the [iFFT] IP and function...
Passed!!
iFFT python execution time: 0.0306615829 sec
iFFT IP execution time:     0.0034842491 sec
Acceleration rate: 8.8
Exit process
